In [9]:
import pandas as pd
import db_connect
import os

In [3]:
earth_weather_data = db_connect.earth_weather_data()

Successfully connected to the database!


In [14]:
daily_average = earth_weather_data.groupby(['date', 'location']).agg({
    "latitude" : "first",
    "longitude" : "first",
    "temperature_2m" : "mean",
    "relative_humidity_2m" : "mean",
    "rain" : "mean",
    "direct_radiation_instant" : "mean"
}).reset_index()


In [6]:
daily_average.head()

,date,latitude,longitude,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant,location
0,1940-01-01 07:00:00,25.0,13.0,-2.023884,66.252353,0.003846,151.680917,True
1,1940-01-01 08:00:00,25.0,13.0,-1.491192,63.522008,0.015385,189.521734,True
2,1940-01-01 09:00:00,25.0,13.0,-0.798884,61.696856,0.011538,202.938687,True
3,1940-01-01 10:00:00,25.0,13.0,0.008808,59.541182,0.026923,206.423082,True
4,1940-01-01 11:00:00,25.0,13.0,0.314577,57.913924,0.011538,170.911665,True


In [9]:
daily_average["location"].unique()

array(['Amazon', 'Athens', 'Everest', 'London', 'McMurdo', 'Oymyakon',
       'Sahara', 'Serengeti', 'Sydney', 'Valley', 'Zurich'], dtype=object)

In [13]:
daily_average.to_parquet('df.parquet.gzip', compression='gzip')


In [10]:
if not os.path.exists('data'):
    os.makedirs('data')

In [11]:
for location in earth_weather_data['location'].unique():
    subset = earth_weather_data[earth_weather_data['location'] == location].groupby('date').agg({
        "latitude": "first",
        "longitude": "first",
        "temperature_2m": "mean",
        "relative_humidity_2m": "mean",
        "rain": "mean",
        "direct_radiation_instant": "mean",
    }).reset_index()
    
    # Save the aggregated data for this location to a Parquet file
    subset.to_parquet(os.path.join('data', f'{location}.parquet.gzip'), index=False, compression="gzip")
